In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = SparkSession.builder.appName('nd').getOrCreate()

In [23]:
data = spark.read.csv(r"C:\Users\Asus\Desktop\studia big data\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Logistic_Regression\customer_churn.csv", inferSchema = True, header=True)

In [24]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [25]:
data.printSchema

<bound method DataFrame.printSchema of DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string, Churn: int]>

In [26]:
prepared_data = data.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',  'Churn'])

In [27]:
null_free = prepared_data.na.drop

In [28]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

In [29]:
from pyspark.sql.functions import countDistinct

In [58]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [59]:
model = assembler.transform(data)


In [60]:
final = model.select('features', 'Churn')
train_data, test_data = final.randomSplit([0.7, 0.3])

In [61]:
from pyspark.ml.classification import LogisticRegression

In [62]:
lr = LogisticRegression(labelCol='Churn')

In [63]:
lr_model = lr.fit(train_data)

In [73]:
training_sum = lr_model.summary

In [74]:
training_sum.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[4.65315773091174...|[0.99055853433502...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[4.89336598791302...|[0.99255962612540...|       0.0|
|[26.0,8787.39,1.0...|  1.0|[0.50047377763874...|[0.62257066424425...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.70071417738344...|[0.99877147799569...|       0.0|
|[27.0,8628.8,1.0,...|  0.0|[5.66456510368655...|[0.99654532109971...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[8.18206942348817...|[0.99972045548376...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.45467029915035...|[0.81071615714475...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.18422472941983...|[0.98499458088639...|       0.0|
|[28.0,11245.38,0....|  0.0|[3.88255134440310...|[0.97981751793324...|       0.0|
|[29.0,8688.17,1

In [76]:
testing = lr_model.evaluate(test_data)

In [83]:
testing.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11204.23,0....|    0|[1.69108250122773...|[0.84436646627746...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.35053505854359...|[0.98726437980817...|       0.0|
|[29.0,13240.01,1....|    0|[6.79803201764769...|[0.99888527480308...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.12797635307839...|[0.99782375631403...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.36516799326981...|[0.98744706011340...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.31236416014472...|[0.96485054702146...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.3166859467403,...|[0.96499682186618...|       0.0|
|[31.0,11297.57,1....|    1|[0.79106824637874...|[0.68806065775865...|       0.0|
|[31.0,12264.68,1....|    0|[3.46983055217581...|[0.96981705884391...|       0.0|
|[32.0,6367.22,1

In [84]:
testing.areaUnderROC

0.9081159420289858

In [87]:
testing.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,11204.23,0....|    0|[1.69108250122773...|[0.84436646627746...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.35053505854359...|[0.98726437980817...|       0.0|
|[29.0,13240.01,1....|    0|[6.79803201764769...|[0.99888527480308...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.12797635307839...|[0.99782375631403...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.36516799326981...|[0.98744706011340...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.31236416014472...|[0.96485054702146...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.3166859467403,...|[0.96499682186618...|       0.0|
|[31.0,11297.57,1....|    1|[0.79106824637874...|[0.68806065775865...|       0.0|
|[31.0,12264.68,1....|    0|[3.46983055217581...|[0.96981705884391...|       0.0|
|[32.0,6367.22,1

In [91]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='Churn')


In [93]:
auc = churn_eval.evaluate(testing.predictions)

In [94]:
auc

0.7695652173913045

In [96]:
new_test = spark.read.csv(r"C:\Users\Asus\Desktop\studia big data\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Logistic_Regression\new_customers.csv", inferSchema = True, header = True)

Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date=datetime.datetime(2011, 8, 29, 18, 37, 54), Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd')

In [99]:
final_lr = lr.fit(final)

In [100]:
testing2 = assembler.transform(new_test)

In [101]:
final_results = final_lr.transform(testing2)

In [104]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

